In [103]:
import dask.bag as db

Helper function that displays all the functions that can be invoked on an object.

In [104]:
def display_object_method(obj):
    object_methods = [method_name for method_name in dir(obj)
                  if callable(getattr(obj, method_name))]
    print(object_methods)

Stand in function for creating the json files.

Will be updated later

In [105]:
def create_json_file(data, filename):
    print(data)
    print(filename)

## Subproject 1
### Top 40 words across all files
- Find top 40 words
- No need to filter out stop words

In [106]:
word_bag = db.read_text(["handout\data\pg36.txt", "handout\data\pg3207.txt"])

In [107]:
stripped_words = word_bag.str.strip()

In [108]:
split_words = stripped_words.str.split(" ")

In [109]:
word_array = split_words.flatten()

In [110]:
lowercase_words = word_array.map(lambda x: x.lower()).filter(lambda x: x != "")

In [111]:
word_frequencies = lowercase_words.frequencies()

In [112]:
object_method_counter(word_frequencies)

['__await__', '__class__', '__dask_graph__', '__dask_keys__', '__dask_layers__', '__dask_optimize__', '__dask_postcompute__', '__dask_postpersist__', '__dask_scheduler__', '__dask_tokenize__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', 'accumulate', 'all', 'any', 'compute', 'count', 'distinct', 'filter', 'flatten', 'fold', 'foldby', 'frequencies', 'groupby', 'join', 'map', 'map_partitions', 'max', 'mean', 'min', 'persist', 'pluck', 'product', 'random_sample', 'reduction', 'remove', 'repartition', 'starmap', 'std', 'sum', 'take', 'to_avro', 'to_dataframe', 'to_delayed', 'to_textfiles', 'topk', 'unzip', 'var', 'visualize']


In [113]:
top40_words = word_frequencies.topk(40, key=1)

In [114]:
word_counts = top40_words.compute()
print(word_counts)

[('the', 19988), ('of', 13365), ('and', 9866), ('to', 8526), ('that', 5383), ('in', 5153), ('a', 4833), ('is', 4687), ('by', 2880), ('for', 2806), ('it', 2553), ('but', 2538), ('as', 2529), ('or', 2481), ('be', 2346), ('not', 2345), ('they', 2002), ('which', 1977), ('his', 1886), ('he', 1883), ('i', 1759), ('was', 1720), ('their', 1711), ('are', 1703), ('from', 1625), ('all', 1255), ('with', 1233), ('have', 1196), ('this', 1149), ('no', 1144), ('there', 1064), ('had', 1020), ('were', 1009), ('any', 932), ('at', 924), ('one', 903), ('them', 847), ('other', 843), ('so', 818), ('on', 774)]
